In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial

In [2]:
def ell(x, w, m):
    return np.stack([0 if j < m else factorial(j)*x**(j-m)/factorial(j-m) for j in range(0, w)])

def C(alpha, n, m, k):
    w = len(alpha)
    M = np.vander(alpha, w, increasing=True)
    M[k] = ell(alpha[k], w, m)
    return ell(alpha[k], w, n) @ np.linalg.inv(M)

In [3]:
C((-2, -1, 0, 1, 2), n=1, m=0, k=2) * 12

array([ 1., -8.,  0.,  8., -1.])

In [4]:
C((-2, -1, 0, 1, 2), n=2, m=0, k=2) * 12

array([ -1.,  16., -30.,  16.,  -1.])

In [8]:
C((-2, -1, 0, 1, 2), n=2, m=1, k=0) * 12

array([-16.8, -36.8,  63.6, -33.6,   6.8])